# DATA COLLECTION
---

Nhiệm vụ phần này: từ danh sách các đường dẫn phim của trang web `Just Watch`, thực hiện cào thông tin **imdbId** trong đường dẫn và dùng **imdbId** đó để cào thông tin của phim đó trong trang web `OMDB` bằng api.

## 1. IMPORT THƯ VIỆN CẦN THIẾT

In [9]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import re

## 2. HÀM CÀO THÔNG TIN IMDBID TRONG TRANG `JUST WATCH`

In [8]:
links = []

with open('../Data/links.txt', 'r') as f:
    links = [line.strip() for line in f]

Dùng selenium thoi vì request k được

In [13]:
# chia 3 ra nha mina hong no k cho minh lay :D
# từ từ để ngâm cú chứ nhìn này trông cx hơi ngu :v
member_part = {
    "Trinh": {
        "part": 1,
        "api_key": None, # đk api xong áp vô nè
        "start": 0,
        "end": 628
    },

    "Thi": {
        "part": 2,
        "api_key": None, # đk api xong áp vô nè
        "start": 808,
        "end": 1258
    },

    "MinhUyen": {
        "part": 3,
        "api_key": 'd4c83dec',
        "start": 1259,
        "end": 1887
    } 
}

In [14]:
# Điền tên mn vào đây theo từ điển nè
info = member_part["MinhUyen"]

part = info["part"]
apiKey = info["api_key"]
start = info["start"]
end = info["end"]

print(part)
print(apiKey)
print(start)
print(end)

3
d4c83dec
1259
1887


In [20]:
def collect_imdbID(links, start, end):
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    time.sleep(10)
    imdbIds = []
    for url in links[start: end]:
        print(f"[LOG]: parse imdbid of url {url}")
        driver.get(url)
        page_source = driver.page_source

        imdb_id_match = re.search(r'"imdbId":"(tt\d+)"', page_source)
        if imdb_id_match:
            imdb_id = imdb_id_match.group(1)
            imdbIds.append(imdb_id)
            # print("IMDB ID:", imdb_id)
        else:
            # print("Không tìm thấy IMDb ID.")
            imdbIds.append("Null")

        time.sleep(3)

    driver.quit()

    return imdbIds

In [21]:
imdbIds = collect_imdbID(links, start=start, end=end)
imdbIds

[LOG]: parse imdbid of url https://www.justwatch.com/us/movie/the-mission-2023
[LOG]: parse imdbid of url https://www.justwatch.com/us/movie/the-peasants
[LOG]: parse imdbid of url https://www.justwatch.com/us/movie/the-burial-0
[LOG]: parse imdbid of url https://www.justwatch.com/us/movie/saw-x
[LOG]: parse imdbid of url https://www.justwatch.com/us/movie/last-call-for-istanbul
[LOG]: parse imdbid of url https://www.justwatch.com/us/movie/coup-de-chance
[LOG]: parse imdbid of url https://www.justwatch.com/us/movie/shooting-stars-2023
[LOG]: parse imdbid of url https://www.justwatch.com/us/movie/wingwomen
[LOG]: parse imdbid of url https://www.justwatch.com/us/movie/love-in-the-maldives
[LOG]: parse imdbid of url https://www.justwatch.com/us/movie/one-day-as-a-lion
[LOG]: parse imdbid of url https://www.justwatch.com/us/movie/freelance-2023
[LOG]: parse imdbid of url https://www.justwatch.com/us/movie/marlowe
[LOG]: parse imdbid of url https://www.justwatch.com/us/movie/la-passion-de-d

['tt28097834',
 'tt10651230',
 'Null',
 'tt21816322',
 'tt23334616',
 'tt15140278',
 'tt3715152',
 'tt21638298',
 'tt26938504',
 'tt15783426',
 'tt15744298',
 'tt6722802',
 'tt19760052',
 'tt11828692',
 'tt7991508',
 'tt2906216',
 'tt10160976',
 'tt21103300',
 'tt26736061',
 'tt21276878',
 'tt12404266',
 'tt6166392',
 'tt26452836',
 'tt10366206',
 'Null',
 'tt21418018',
 'Null',
 'tt25154848',
 'tt15246162',
 'Null',
 'tt28856894',
 'tt27800144',
 'Null',
 'tt21239752',
 'tt26215632',
 'tt15206296',
 'Null',
 'tt28338336',
 'tt30252260',
 'tt29362834',
 'tt27496114',
 'tt13080852',
 'tt26742022',
 'tt27684092',
 'tt32308707',
 'tt27698813',
 'tt27244707',
 'tt26734180',
 'Null',
 'tt24484852',
 'Null',
 'tt27549751',
 'tt21953278',
 'tt16986596',
 'tt28248955',
 'tt14237412',
 'tt23538910',
 'tt30826209',
 'Null',
 'tt30781889',
 'Null',
 'tt30575595',
 'tt13869038',
 'tt21398920',
 'Null',
 'tt32422589',
 'tt15464420',
 'tt5660834',
 'tt26741816',
 'tt14399126',
 'tt24853090',
 'tt279

## 3. HÀM LẤY DỮ LIỆU TỪ API CỦA TRANG `OMDB`

In [ ]:
def collectMovieData(imdbIds, key = 0):
    title = []
    rated = []
    released = []
    runtime = []
    genre = []
    metaScore = []
    rating = []
    vote = []
    boxCffice = []

    for id in imdbIds:
        print(f"[LOG]: Collecting data of IMDbId {id}")

        url = f'https://www.omdbapi.com/?i={id}&apikey={key}'
        trying = 1

        while True:    
            respond = requests.get(url)
            if respond.status_code == 200:
                data = respond.json()
                title.append(data['Title'])
                rated.append(data['Rated'])
                released.append(data['Released'])
                runtime.append(data['Runtime'])
                genre.append(data['Genre'])
                metaScore.append(data['Metascore'])
                rating.append(data['imdbRating'])
                vote.append(data['imdbVotes'])
                boxCffice.append(data['BoxOffice'])
                break
                
            else:
                trying += 1
                if trying > 3:
                    print(f"Request fail after 3 retries with code {respond.status_code}")

                time.sleep(1)

    movieDataframe = pd.DataFrame({
            "Title": title,
            "Rated": rated,
            "Released": released,
            "Runtime": runtime,
            "Genre": genre,
            "Metascore": metaScore,
            "IMDb Rating": rating,
            "IMDb Votes": vote,
            "Domestic Revenue": boxCffice

        })

    return movieDataframe

In [ ]:
df1 = collectMovieData(imdbIds, key=apiKey)
df1

In [22]:
print(os.path.join("./Data/", f"raw_movie_data_{str(part)}.csv"))

./Data/raw_movie_data_3.csv


In [ ]:
# Lưu file đây nhé
file_name = os.path.join("./Data/", f"raw_movie_data_{str(part)}.csv")
df1.to_csv(file_name)